In [2]:
import numpy as np
import pandas as pd
from scipy.integrate import solve_ivp
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
from iprocessor import add_day_name_column, add_date_name_column, smooth_sundays_rolling_w7_l

In [3]:
def get_value_by_date(input_date, input_dataframe, column_name):
    """
    Get the value from the specified column in the DataFrame based on the provided date.

    Parameters:
    - input_date (str): The date to look for in the DataFrame.
    - input_dataframe (pd.DataFrame): The DataFrame containing the data.
    - column_name (str): The column from which to extract the value.

    Returns:
    - value: The value from the specified column corresponding to the given date.
    """
    try:

        filtered_df = input_dataframe[input_dataframe['Date'] == input_date]

        # Extract the value from the specified column
        value = filtered_df[column_name].iloc[0]

        return value

    except IndexError:
        print(f"No data available for the date {input_date} in the DataFrame.")
        return None

In [5]:
df = pd.read_csv(r'data_csv\German_case.csv')

In [7]:
# Initialize date
init_date = pd.to_datetime('2020-05-01')

# Check if the chosen init_date exists in the dataframe
if init_date < df['Date'].min() or init_date > df['Date'].max():
    raise ValueError('The date is out of the range of the dataframe')

# Initialization of the compartments
date_init = init_date
date_ec = init_date + pd.Timedelta(days=exposed_period + asymptomatic_period)
date_c = init_date + pd.Timedelta(days=asymptomatic_period)
date_i = init_date - pd.Timedelta(days=infectious_period)
date_if = init_date - pd.Timedelta(days=isolated_period + infectious_period)

# Ensure prob_asymptomatic is not 1 to avoid division by zero
if prob_asymptomatic == 1:
    raise ValueError("Probability of asymptomatic cases cannot be 1")

E0 = (1 / (1 - prob_asymptomatic)) * (get_value_by_date(date_ec, df_observed, 'Confirmed') - get_value_by_date(date_c, df_observed, 'Confirmed'))
A0 = (1 / (1 - prob_asymptomatic)) * (get_value_by_date(date_c, df_observed, 'Confirmed') - get_value_by_date(date_init, df_observed, 'Confirmed'))
I0 = prob_quarant_inf * (get_value_by_date(date_i, df_observed, 'Confirmed') - get_value_by_date(date_if, df_observed, 'Confirmed'))
F0 = get_value_by_date(init_date, df_observed, 'Confirmed') - get_value_by_date(date_i, df_observed, 'Confirmed')
R0 = get_value_by_date(init_date, df_observed, 'Recovered')
D0 = get_value_by_date(init_date, df_observed, 'Deaths')
S0 = total_population - E0 - A0 - I0 - R0 - D0 - F0

print(f"E0: {E0}, A0: {A0}, I0: {I0}, F0: {F0}, R0: {R0}, D0: {D0}, S0: {S0}")

TypeError: '<' not supported between instances of 'Timestamp' and 'str'